In [1]:
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders
from manipulation.scenarios import MakeManipulationStation

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

model_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/urdf/planar_iiwa14_spheres_dense_elbow_collision.urdf
    default_joint_positions:
        iiwa_joint_2: [0.1]
        iiwa_joint_4: [-1.2]
        iiwa_joint_6: [1.6]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
"""

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
def teleop_2d_direct(interactive=False, q_cmd=np.array([0, 2, 3])):
    """
    Joint position control of the Kuka iiwa robot, without using teleop sliders or differential IK.

    Args:
        interactive (bool): If True, function will query the user to manually set the desired joint positions
            while running the simulator. Otherwise, function will use "q_cmd" as the target joint position.
        q_cmd (np.ndarray): Shape (3,). Desired positions of the three movable joints on the 2D robot.
            "q_cmd" cannot be None if "interactive" is set to False.
    """
    assert (not interactive and q_cmd is not None) or interactive, 'Variable "q_cmd" must not be None if the function is run in non-interactive mode'

    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step))

    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, 
        station.GetOutputPort("query_object"),
        meshcat)
    meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)

    #######################################################################
    # Your code here 
    # (setup the diagram, simulation, and necessary contexts)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)

    #######################################################################

    if simulator is None:
        print("You must set the simulator variable above")
        return station, context
        
    simulator.set_target_realtime_rate(1.0 if interactive else 0)
    meshcat.AddButton("Stop Simulation")
    while meshcat.GetButtonClicks("Stop Simulation") < 1:
        simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)       

        #######################################################################
        # Your code here 
        # (read the current measured joint angles into the variable `q_current`)
        # (hint: what output ports does the `station` instance have available?)
        
        q_current = station.GetOutputPort('iiwa_position_measured').Eval(station_context)
        print(f'Current joint angles: {q_current}')            

        ####################################################################### 

        if interactive and running_as_notebook:
            j1_input = input(f'Please enter value for first movable joint (current value: {q_current[0]:.3f})')
            j2_input = input(f'Please enter value for second movable joint (current value: {q_current[1]:.3f})')
            j3_input = input(f'Please enter value for third movable joint (current value: {q_current[2]:.3f})') 
            j_inputs = [j1_input, j2_input, j3_input]
            q_cmd = q_current.copy()
            for j_idx, j_inp in enumerate(j_inputs):
                try:
                    j_val = float(j_inp)
                except Exception as e:
                    print(e)
                    print(f'Setting joint {j_idx} value to current joint value')
                    j_val = q_cmd[j_idx]
                q_cmd[j_idx] = j_val                      

        #######################################################################
        # Your code here 
        # (command the desired joint positions, and read the joint angle command into variable `q_current_cmd`)
        
        station.GetInputPort('iiwa_position').FixValue(station_context, q_cmd)
        q_current_cmd = station.GetOutputPort('iiwa_position_commanded').Eval(station_context)
        print(f'Current commanded joint angles: {q_current_cmd}\n')

        #######################################################################  

        if not interactive or not running_as_notebook:
            break
            
    meshcat.DeleteButton("Stop Simulation")
        
    return station, context

teleop_2d_direct(True);

Current joint angles: [-0.00027473  0.00180615 -0.00333018]
Please enter value for first movable joint (current value: -0.000)0.4
Please enter value for second movable joint (current value: 0.002)0
Please enter value for third movable joint (current value: -0.003)0
Current commanded joint angles: [0.4 0.  0. ]

Current joint angles: [0.40204031 0.02194281 0.00483079]
Please enter value for first movable joint (current value: 0.402)0.4
Please enter value for second movable joint (current value: 0.022)0.5
Please enter value for third movable joint (current value: 0.005)0.0
Current commanded joint angles: [0.4 0.5 0. ]

Current joint angles: [ 0.38320092  0.46321063 -0.02320342]
Please enter value for first movable joint (current value: 0.383)0.4
Please enter value for second movable joint (current value: 0.463)0.4
Please enter value for third movable joint (current value: -0.023)-0.6
Current commanded joint angles: [ 0.4  0.4 -0.6]

Current joint angles: [ 0.38604068  0.37541371 -0.59706

In [20]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.robot.test_direct_joint_control import \
    TestDirectJointControl

Grader.grade_output([TestDirectJointControl], [locals()], 'results.json')
Grader.print_test_results('results.json')

Total score is 5/5.

Score for Test iiwa_position_measured is 5/5.
- Current joint angles: [-0.00027473  0.00180615 -0.00333018]
Current commanded joint angles: [0.1 0.2 0.3]


